In [51]:
from preprocess import Preprocess
from urllib.request import urlopen
import timm
import torch
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [52]:
TRANSFORM = transforms.ToTensor()

In [54]:
class TIMM:
    def __init__(self):
        # https://huggingface.co/timm/vit_wee_patch16_reg1_gap_256.sbb_in1k
        self.model = timm.create_model(
            'vit_wee_patch16_reg1_gap_256.sbb_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
        self.model.eval()

        # get model specific transforms (normalization, resize)
        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=False)

    @functools.lru_cache(maxsize=None)
    def extract(self, img) -> torch.Tensor:
        with torch.no_grad():
            return self.model(self.transforms(img).unsqueeze(0))[0]

timm_model = TIMM()

In [56]:
# transforma cada par em uma linha de uma ndarray (é quem criou os CSVs dev_train e dev_val)
def pairs_as_ndarray(pairs):
    result = []
    for i, pair in enumerate(pairs):
        image1, image2, label = pair
        print(f'Progress: {i}/{len(pairs)}         ', end='\r')
        attr1 = timm_model.extract(image1)
        attr2 = timm_model.extract(image2)
        row = np.concatenate((attr1, attr2, [label]))
        result.append(row)
    return np.array(result)

In [57]:
dev_train = pairs_as_ndarray(Preprocess.load_train_pairs(transform=TRANSFORM))
dev_val = pairs_as_ndarray(Preprocess.load_test_pairs(transform=TRANSFORM))
# Load from .npy
# dev_train = np.load('dev_train.npy')
# dev_val = np.load('dev_val.npy')
dev_train.shape, dev_val.shape

((2200, 513), (1000, 513))

In [58]:
# Save the arrays
np.save('dev_train.npy', dev_train)
np.save('dev_val.npy', dev_val)

In [59]:
X_train = dev_train[:, :-1]
y_train = dev_train[:, -1]
X_val = dev_val[:, :-1]
y_val = dev_val[:, -1]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2200, 512), (2200,), (1000, 512), (1000,))

In [60]:
### Regressão logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 0.0031622776601683794, 'solver': 'sag'}
Best cross-validation score:  0.5345454545454544


In [61]:

logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))


Train accuracy: 0.6827272727272727
Val accuracy: 0.555


In [62]:

### Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)


Best parameters:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': None}
Best cross-validation score:  0.6704545454545455


In [63]:

rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))


Train accuracy: 0.9990909090909091
Val accuracy: 0.697


In [68]:

### Rede neural
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs',
    project_name=f'nn-timm-wee')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]


Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.7222000002861023

Best val_accuracy So Far: 0.7247999906539917
Total elapsed time: 00h 02m 22s
Results summary
Results in runs\nn-timm-wee
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 16
learning_rate: 0.01
units_1: 28
units_2: 4
Score: 0.7247999906539917

Trial 4 summary
Hyperparameters:
num_layers: 1
units_0: 16
learning_rate: 0.01
units_1: 16
units_2: 16
Score: 0.7222000002861023

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 32
learning_rate: 0.01
units_1: 4
Score: 0.721399986743927

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 24
learning_rate: 0.01
units_1: 4
units_2: 24
Score: 0.7190000057220459

Trial 2 summary
Hyperparameters:
num_layers: 1
units_0: 16
learning_rate: 0.0001
units_1: 20
units_2: 4
Score: 0.7110000014305115


In [69]:

best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()


Epoch 1/40


69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5540 - loss: 0.6812 - val_accuracy: 0.6170 - val_loss: 0.6394
Epoch 2/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6577 - loss: 0.5976 - val_accuracy: 0.6090 - val_loss: 0.7057
Epoch 3/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6782 - loss: 0.5772 - val_accuracy: 0.6390 - val_loss: 0.6668
Epoch 4/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7300 - loss: 0.5234 - val_accuracy: 0.6200 - val_loss: 0.6667
Epoch 5/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7323 - loss: 0.5161 - val_accuracy: 0.6590 - val_loss: 0.7342
Epoch 6/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - accuracy: 0.7968 - loss: 0.4548 - val_accuracy: 0.6690 - val_loss: 0.7329
Epoch 7/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8031 - loss: 0.4338 - val_accuracy: 0.6780 - val_loss: 0.7310
Epoch 8/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8265 - loss: 0.3928 - val_accuracy: 0.7170 - val_loss: 0.6882
E

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 16)             │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28)             │           476 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │           116 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,417 (103.20 KB)

 Trainable params: 8,805 (34.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,612 (68.80 KB)

In [70]:

train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - accuracy: 0.9925 - loss: 0.0620
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.7142 - loss: 2.9136


(0.9763636589050293, 0.703000009059906)

In [71]:


### SVM
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score:  0.7086363636363637


In [72]:

# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 0.8754545454545455
Val accuracy: 0.711
